In [6]:
import oqd_compiler_infrastructure as ci
from trical.backend.qutip.rule import time_evolve
import numpy as np
from math import sqrt
from typing import List, Tuple

class Ion(ci.TypeReflectBaseModel):
    name: str
    energy_levels: List[tuple]

class Ca40(Ion):
    def __init__(self, energy_levels):
        super().__init__(name="Ca40", energy_levels=energy_levels)

    def transition(self, states):
        return states  

class Chain(ci.TypeReflectBaseModel):
    ions: List[Ion]
    trap_freqs: List[float]
    selected_modes: List[int]
    modes: List[int] = [0]  

class Laser(ci.TypeReflectBaseModel):
    phi: float
    k_hat: Tuple[float, float, float]
    eps_hat: Tuple[float, float, float]
    wavelength: float = None
    intensity: float = None

class Chamber(ci.TypeReflectBaseModel):
    chain: Chain
    lasers: List[Laser]
    B: float
    Bhat: Tuple[float, float, float]

    def set_laser_wavelength_from_transition(self, laser_index, transition):
        print(f"Setting laser {laser_index} wavelength based on transition {transition}")

    def set_laser_intensity_from_pi_time(self, laser_index, pi_time, transition):
        print(f"Setting laser {laser_index} intensity based on pi_time {pi_time}")

def construct_H_tree(chamber, timescale):
    print(f"Constructing Hamiltonian tree with timescale {timescale}")
    return "Hamiltonian Tree"

ion = Ca40([("S1/2", -1/2, "S"), ("D5/2", -1/2, "D")])
chain = Chain(ions=[ion], trap_freqs=[3e6, 3e6, 1e6], selected_modes=[])

mode = chain.modes[-1]
chain.selected_modes = [mode]

laser = Laser(
    phi=0,
    k_hat=tuple(np.array([1, 0, 1]) / sqrt(2)),
    eps_hat=tuple(np.array([1, 0, -1]) / sqrt(2))
)

chamber = Chamber(
    chain=chain,
    lasers=[laser],
    B=4e-4,
    Bhat=(0, 0, 1)
)

chamber.set_laser_wavelength_from_transition(0, ion.transition(("S", "D")))
chamber.set_laser_intensity_from_pi_time(0, 2e-6, ion.transition(("S", "D")))

timescale = 1e-6
H_tree = construct_H_tree(chamber, timescale)

printer = ci.Post(ci.PrettyPrint())
print(printer(H_tree))


Setting laser 0 wavelength based on transition ('S', 'D')
Setting laser 0 intensity based on pi_time 2e-06
Constructing Hamiltonian tree with timescale 1e-06
str(Hamiltonian Tree)


In [2]:
from qutip import tensor
import numpy as np


duration = 10
times = np.linspace(0, duration,  1000)

psi_ion = ion.state["S"]
psi_mot = mode.setstate(0)

psi_0 = tensor(psi_ion, psi_mot)

op = chain.ion_projector(1, "D")

results = time_evolve(H_tree, psi_0, times, expt_ops=[op], progress_bar= True)
results.quick_plot()


NameError: name 'ion' is not defined

In [14]:
backend = QutipBackend()

compiled_prog = backend.compile(chamber, args)

results = backend.run(program=compiled_prog, args =args)


Updated sys.path: ['/Users/rivajangra/.qutip/qutip_coeffs_1.1', '/Users/rivajangra/Desktop/OQD/TrICal/tutorial', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/rivajangra/Library/Python/3.10/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages', '/Users/rivajangra/Desktop/OQD/TrICal/tutorial/src']
